# Backtest

In [1]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

agentIds = [1, 2]
env.config_agents(agentIds)

params_1 = {"agentId" : 1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}
params_2 = {"agentId" : 2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}
params_3 = {"agentId" : 3, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"MOMENTUM"}
params_4 = {"agentId" : 4, "symbol" : "ETH", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

In [2]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

agent_3 = Agent(**params_3)
agent_3.update_policy_params(params = 5)
agent_3.run_episode()

agent_4 = Agent(**params_4)
agent_4.update_policy_params(params = (7, 14, 21))
agent_4.run_episode()

In [3]:
env.get_report(agentId=1, symbol = "BTC")

## Global Report

#  MultiAgent System

In [4]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }
env = Env(**config)


def custom_action():
    print(" *** Barrier passee *** ")


condition = Condition()
barrier = Barrier(2, action=custom_action)

agentId_1 = 1
agentId_2 = 2

agent_msg = {agentId_1 : None, agentId_2 : None}
master_msg = {agentId_1 : None, agentId_2 : None}

activeAgent = 1

boss_params = {"condition" : condition, "activeAgent" : activeAgent ,  "agent_msg" : agent_msg, "master_msg" : master_msg}

params_1 = {"agentId" : agentId_1, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}

params_2 = {"agentId" : agentId_2, "symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA",
            "condition" : condition, "barrier" : barrier, "agent_msg" : agent_msg, "master_msg" : master_msg}


Boss = MasterAgentThread(**boss_params)
Boss.addAgent(1)
Boss.addAgent(2)

agent_1 = MAgentThread(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
Boss.start()
agent_1.start()
agent_2.start()


agent_1.join()
agent_2.join()
Boss.join()

**** Master ****
Master a tous les agents , Activez-vous
Waiting for agents ... 
1 en cours d'execution ... 
======> Place an Order ... 
___ msg : Place an Order ... 
1 - date : 2023-02-14 00:00:00, BTC
2 en cours d'execution ... 
======> Place an Order ... 
___ msg : Place an Order ... 
2 - date : 2023-02-14 00:00:00, BTC
 *** Barrier passee *** 
 ooo__oooo=> : {1: 'order placed ... ', 2: 'order placed ... '}
stop condition : False
**** Master ****
Master a tous les agents , Activez-vous
Waiting for agents ... 
1 en cours d'execution ... 
======> Place an Order ... 
___ msg : Place an Order ... 
1 - date : 2023-02-15 00:00:00, BTC
2 en cours d'execution ... 
======> Place an Order ... 
___ msg : Place an Order ... 
2 - date : 2023-02-15 00:00:00, BTC
 *** Barrier passee *** 
 ooo__oooo=> : {1: 'order placed ... ', 2: 'order placed ... '}
stop condition : False
**** Master ****
Master a tous les agents , Activez-vous
Waiting for agents ... 
1 en cours d'execution ... 
======> Place an 

In [6]:
trades = env.journal.trades_data.copy()
trades.head()

,agentId,date,price,quantity,position,side,state,in_value,out_value,value,pnl,pnl_pct,symbol
0,1,2023-02-14,22199.84,0.004505,1,LONG,"('Open', 'LONG', False, False)",100.0,0.0,100.000000,0.000000,0.000000,BTC
1,2,2023-02-14,22199.84,0.004505,1,LONG,"('Open', 'LONG', False, False)",100.0,0.0,100.000000,0.000000,0.000000,BTC
2,1,2023-02-15,24324.05,0.004505,1,LONG,"('-', 'LONG', False, False)",100.0,0.0,109.568582,9.568582,0.095686,BTC
3,2,2023-02-15,24324.05,0.004505,1,LONG,"('-', 'LONG', False, False)",100.0,0.0,109.568582,9.568582,0.095686,BTC
4,1,2023-02-16,23517.72,0.004505,1,LONG,"('-', 'LONG', False, False)",100.0,0.0,105.936439,5.936439,0.059364,BTC


In [7]:
dd = env.journal.trades_data
dd.head

<bound method NDFrame.head of      agentId       date     price  quantity  position   side  \
0          1 2023-02-14  22199.84  0.004505         1   LONG   
1          2 2023-02-14  22199.84  0.004505         1   LONG   
2          1 2023-02-15  24324.05  0.004505         1   LONG   
3          2 2023-02-15  24324.05  0.004505         1   LONG   
4          1 2023-02-16  23517.72  0.004505         1   LONG   
..       ...        ...       ...       ...       ...    ...   
501        2 2023-10-22  29992.46  0.000000        -1  SHORT   
502        1 2023-10-23  33069.99  0.004639         1   LONG   
503        2 2023-10-23  33069.99  0.000000        -1  SHORT   
504        1 2023-10-24  34024.90  0.004639         1   LONG   
505        2 2023-10-24  34024.90  0.000000        -1  SHORT   

                              state    in_value  out_value       value  \
0    ('Open', 'LONG', False, False)  100.000000        0.0  100.000000   
1    ('Open', 'LONG', False, False)  100.000000      

In [8]:
import plotly.figure_factory as ff

dd1 = dd[dd["agentId"] == 1]
dd2 = dd[dd["agentId"] == 2]


# Group data together
hist_data = [dd1["pnl"], dd2["pnl"]]

group_labels = ['Agent 1', 'Agent 2']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [9]:
len(dd2)

253

In [10]:
len(dd1)

253

In [11]:
from plotly.figure_factory import create_distplot
import numpy as np
import pandas as pd

df = pd.DataFrame({'2012': np.random.randn(200), 
                   '2013': np.random.randn(200)+1}
                  )
fig = create_distplot([df[c] for c in df.columns], df.columns)
fig.show()

In [12]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)

x = np.random.randn(1000)
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [13]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = np.random.randn(200) - 2
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 2
x4 = np.random.randn(200) + 4

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [14]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200)
x2 = np.random.randn(200) + 2

group_labels = ['Group 1', 'Group 2']

colors = ['slategray', 'magenta']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.5,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

In [15]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200) - 1
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 1

hist_data = [x1, x2, x3]

group_labels = ['Group 1', 'Group 2', 'Group 3']
colors = ['#333F44', '#37AA9C', '#94F3E4']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text='Curve and Rug Plot')
fig.show()

In [16]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()